In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statistics
%matplotlib inline
from datetime import datetime
from pandas import Series
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

import seaborn as sns
import os
import statistics

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import TimeSeriesSplit

from dateutil.relativedelta import relativedelta 
from scipy.optimize import minimize              

import statsmodels.formula.api as smf          
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product      

from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing, Holt

from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams

from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [3]:
from math import sqrt

def rmse(y_true, y_pred):
    squared_error = 0
    for i in range(len(y_true)):
        squared_error = squared_error + (y_true[i] - y_pred[i]) ** 2
    root_mean_squared_error = sqrt(squared_error / len(y_true))
    return root_mean_squared_error

In [4]:
def moving_avg(rounds):
    y_hat['avg'] = 0
    for i in range(len(Valid)):
        y_pred = 0
        for j in range(rounds):
            y_pred = y_pred + train.footfall[len(Train) - (j + 1) * len(Valid) + i]
        y_hat['avg'][i] = y_pred / rounds

In [5]:
def test_stationary(timeseries):
    #Determine rolling statistics
    rolmean = pd.Series(timeseries).rolling(window = 28).mean()
    rolstd = pd.Series(timeseries).rolling(window = 28).std()
    
    #Plot rolling Statistics
    orig = plt.plot(timeseries, color = "blue", label = "Original")
    mean = plt.plot(rolmean, color = "red", label = "Rolling Mean")
    std = plt.plot(rolstd, color = "black", label = "Rolling Std")
    plt.legend(loc = "best")
    plt.title("Rolling Mean and Standard Deviation")
    plt.show(block = False)
    
    #Perform Dickey Fuller test
    print("Results of Dickey Fuller test: ")
    dftest = adfuller(timeseries, autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistics', 'p-value', '# Lag Used', 'Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)' %key] = value
    print(dfoutput)

### Exclude Locations with Too Much Missing Data:

In [6]:
ff = pd.read_csv('stackfootfall.csv')
ff.timestamp = pd.to_datetime(ff.timestamp, format = '%Y-%m-%d %H:%M:%S') 

In [17]:
ff = ff.loc[(ff.timestamp >= '2018-01') & (ff.timestamp <= '2019-08-18 23:00')]

ff_count = ff.groupby('location')['footfall'].count() / max(ff.groupby('location')['footfall'].count())
limit1 = ff_count.loc[ff_count >= 0.6].to_frame()
ff_test = ff.loc[(ff.timestamp >= '2018-07-29') & (ff.timestamp <= '2019-08-18 23:00')]
ff_test_count = ff_test.groupby('location')['footfall'].count() / max(ff_test.groupby('location')['footfall'].count())
limit2 = ff_test_count.loc[ff_test_count >= 0.8].to_frame()
limit = pd.merge(limit1, limit2, how = 'inner', on = 'location')

ff = pd.merge(ff, limit, how = 'inner', on = 'location')

,Unnamed: 0,timestamp,location,device,footfall,year,month,day,hour,day_of_week
0,0,2018-01-05 00:00:00,0.0,1240.0,NaN,2018,1,5,0,5
1,1,2018-01-05 01:00:00,0.0,1240.0,NaN,2018,1,5,1,5
2,2,2018-01-05 02:00:00,0.0,1240.0,NaN,2018,1,5,2,5
3,3,2018-01-05 03:00:00,0.0,1240.0,NaN,2018,1,5,3,5
4,4,2018-01-05 04:00:00,0.0,1240.0,NaN,2018,1,5,4,5
...,...,...,...,...,...,...,...,...,...,...
8896171,260179,2019-08-18 19:00:00,1228.0,1074.0,310.0,2019,8,18,19,7
8896172,260180,2019-08-18 20:00:00,1228.0,1074.0,137.0,2019,8,18,20,7
8896173,260181,2019-08-18 21:00:00,1228.0,1074.0,120.0,2019,8,18,21,7
8896174,260182,2019-08-18 22:00:00,1228.0,1074.0,46.0,2019,8,18,22,7


In [20]:
result = pd.DataFrame(columns = ('location','Naive','Moving_Average12','Moving_Average24',
                                 'Moving_Average30','Expoential_Smoothing','SARIMA'))

In [ ]:
for i in ff.location:
     
    ff_i = ff.loc[(ff.location <= i) & (ff.location >= i)][[
        'timestamp','footfall','year','month','day','hour','day_of_week']]    
    ff_i.index = ff_i.timestamp
    ff_i = ff_i.resample('H').mean()
    
    ff_i['footfall'].replace(0, np.nan, inplace = True)
    ff_i['footfall'] = ff_i['footfall'].interpolate(method='linear')
    ff_i = ff_i.reset_index(level = ['timestamp'])
    ff_i.timestamp = pd.to_datetime(ff_i.timestamp, format = '%Y-%m-%d %H:%M:%S')
    
    temp = ff_i[['timestamp', 'footfall']]
    temp.index = temp.timestamp
    temp['year'] = temp.timestamp.dt.year
    temp['month'] = temp.timestamp.dt.month
    temp['day'] = temp.timestamp.dt.day
    temp['hour'] = temp.timestamp.dt.hour
    temp['day_of_week'] = temp.timestamp.dt.dayofweek + 1
    
    Train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')]
    Valid = temp.loc[(temp.timestamp >= '2019-07-29 00:00') & (temp.timestamp <= '2019-08-18 23:00')]
    
    train = Train.resample('D').mean()
    valid = Valid.resample('D').mean() 
    
    train['footfall'] = train['footfall'].interpolate(method='linear')
    valid['footfall'] = valid['footfall'].interpolate(method='linear')

    # Monday to Thursday Ratio
    mtt = Train.loc[(Train.day_of_week >= 1) & (Train.day_of_week <= 4)].groupby('hour')['footfall'].mean().to_frame()
    mtt['ratio'] = mtt['footfall']/mtt['footfall'].sum()
    mtt = mtt.reset_index(level = ['hour'])

    # Friday Ratio
    fri = Train.loc[(Train.day_of_week >= 5) & (Train.day_of_week <= 5)].groupby('hour')['footfall'].mean().to_frame()
    fri['ratio'] = fri['footfall']/fri['footfall'].sum()
    fri = fri.reset_index(level = ['hour'])

    # Sat Ratio
    sat = Train.loc[(Train.day_of_week >= 6) & (Train.day_of_week <= 6)].groupby('hour')['footfall'].mean().to_frame()
    sat['ratio'] = sat['footfall']/sat['footfall'].sum()
    sat = sat.reset_index(level = ['hour'])

    # Sun Ratio
    sun = Train.loc[(Train.day_of_week >= 7) & (Train.day_of_week <= 7)].groupby('hour')['footfall'].mean().to_frame()
    sun['ratio'] = sun['footfall']/sun['footfall'].sum()
    sun = sun.reset_index(level = ['hour'])

    ratio1 = mtt[['hour','ratio']]
    ratio1['day_of_week'] = 1
    ratio2 = mtt[['hour','ratio']]
    ratio2['day_of_week'] = 2
    ratio3 = mtt[['hour','ratio']]
    ratio3['day_of_week'] = 3
    ratio4 = mtt[['hour','ratio']]
    ratio4['day_of_week'] = 4
    ratio5 = fri[['hour','ratio']]
    ratio5['day_of_week'] = 5
    ratio6 = sat[['hour','ratio']]
    ratio6['day_of_week'] = 6
    ratio7 = sun[['hour','ratio']]
    ratio7['day_of_week'] = 7

    ratio_week = ratio1.append(ratio2).append(ratio3).append(ratio4).append(ratio5).append(ratio6).append(ratio7)

    merge = pd.merge(Valid, ratio_week, on = ('hour','day_of_week'), how = 'left')

    # Naive Approach
    dd = np.asarray(Train['footfall'])
    y_hat = Valid.copy()
    y_hat['naive']= dd[len(dd)- 1]
    for i in range(len(Valid)): 
        y_hat['naive'][i]= dd[len(dd) - len(Valid) + i]
    naive_rmse = rmse(Valid.footfall, y_hat.naive)
    
    # Moving Average Approach
    moving_avg(4)
    avg4_rmse = rmse(Valid.footfall, y_hat.avg)
    moving_avg(8)
    avg8_rmse = rmse(Valid.footfall, y_hat.avg)
    moving_avg(10)
    avg10_rmse = rmse(Valid.footfall, y_hat.avg)
    
    # Exponential Smoothing
    y_hat = valid.copy()
    fit1 = ExponentialSmoothing(np.asarray(train.footfall), seasonal_periods = 7, trend = 'add', seasonal= 'add').fit()
    y_hat['Holt_Winter'] = fit1.forecast(len(valid))    
    HoltWinter_rmse = rmse(valid.footfall, y_hat.Holt_Winter)
    
    pred_d = y_hat.Holt_Winter.to_frame()
    pred_d = pred_d.reset_index(level = ['timestamp'])
    pred_d['month'] = pred_d.timestamp.dt.month
    pred_d['day'] = pred_d.timestamp.dt.day
    temp = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
    temp['prediction'] = temp['ratio'] * temp['Holt_Winter'] * 24
    holt_winter_rmse = rmse(temp['footfall'], temp['prediction'])
    
    # SARIMA
    fit1 = sm.tsa.statespace.SARIMAX(train.footfall, order = (1,1,3), seasonal_order =(1,1,3,7)).fit()
    y_hat['SARIMA'] = fit1.predict(start = "2019-07-29 00:00", end = "2019-08-18 23:00", dynamic=True)
    SARIMA_rmse = rmse(valid['footfall'], y_hat['SARIMA'])
    
    pred_d = y_hat['SARIMA'].to_frame()
    pred_d = pred_d.reset_index(level = ['timestamp'])
    pred_d['month'] = pred_d.timestamp.dt.month
    pred_d['day'] = pred_d.timestamp.dt.day
    temp2 = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
    temp2['prediction'] = temp2['ratio'] * temp2['SARIMA'] * 24
    sarima_rmse = rmse(temp2['footfall'], temp2['prediction'])
    
#    result.location[j] = i
#    result.Naive[j] = naive_rmse
#    result.Moving_Average12[j] = avg4_rmse
#    result.Moving_Average24[j] = avg8_rmse
#    result.Moving_Average30[j] = avg10_rmse
#    result.Expoential_Smoothing[j] = holt_winter_rmse
#    result.SARIMA[j] = sarima_rmse
    result = result.append([i, naive_rmse, avg4_rmse, avg8_rmse, avg10_rmse, holt_winter_rmse, sarima_rmse],
                           ignore_index=True)

In [ ]:
result

In [ ]:
result.to_csv('BatchOperation.csv')